# 基于langchain创建自己专属的对话大模型

1. 领域精准问答
2. 数据更新频繁
3. 生成内容可解释可追溯
4. 数据隐私保护

通过这个例子，我们将基于`LangChain`, `OpenAI(LLM)`,  `vector DB`构建一个属于自己的LLM模型。

主要使用的技术————***Retrieval Augmented Generation (RAG)***

首先确保自己拥有一个 `API key` (也并非必须)，此处用的是[阿里的百炼千问](https://www.alibabacloud.com/help/zh/model-studio/get-api-key)，点击遵照官方文档流程获取 API Key。


### 准备环境

In [1]:
import sys
!{sys.executable} -m pip install --upgrade dashscope

  Using cached dashscope-1.24.9-py3-none-any.whl.metadata (7.1 kB)
Using cached dashscope-1.24.9-py3-none-any.whl (1.3 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### 创建一个对话模型(no RAG)

此处建议申请 API 后，把 API 写入环境变量。具体如何写入参考阿里官方文档：[如何设置官方文档](https://www.alibabacloud.com/help/zh/model-studio/configure-api-key-through-environment-variables?spm=a2c63.p38356.0.i1#6ba7c2825a145)。本文将此环境变量写入系统变量，方便任何环境调用。

In [12]:
from langchain_openai import ChatOpenAI
import os

chat = ChatOpenAI(
    model="qwen-plus",   # 或 qwen-turbo / qwen-max
    api_key=os.environ["DASHSCOPE_API_KEY"],
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1", #此处链接为国际版的，国内版本换为https://dashscope.aliyuncs.com
    temperature=0
)

resp = chat.invoke("温度是什么？")
print(resp.content)

温度是表示物体冷热程度的物理量，从微观角度来说，它反映了物体内部分子或原子热运动的剧烈程度。温度越高，分子的平均动能越大，运动越剧烈；温度越低，分子的平均动能越小，运动越缓慢。

### 温度的几个关键点：

1. **宏观定义**：  
   温度是描述系统热平衡状态的物理量。当两个物体接触并达到热平衡时，它们的温度相等。

2. **微观解释**：  
   从统计物理学角度看，温度与物质内部粒子（如分子、原子）的无规则热运动有关。温度正比于这些粒子的平均平动动能。

3. **测量单位**：
   - **摄氏度（°C）**：日常生活中常用，水的冰点为0°C，沸点为100°C（标准大气压下）。
   - **开尔文（K）**：国际单位制中的温度单位，用于科学计算。0 K 是“绝对零度”，即理论上粒子热运动完全停止的状态。0 K = -273.15°C。
   - **华氏度（°F）**：主要用于美国，水的冰点为32°F，沸点为212°F。

4. **绝对零度**：  
   0 K（-273.15°C）是温度的下限，无法真正达到，但可以无限接近。在该温度下，理想气体的分子动能趋近于零。

5. **温度与热量的区别**：  
   - **温度** 表示冷热程度，反映的是分子平均动能。
   - **热量** 是能量的一种形式，表示在热传递过程中转移的能量多少。

### 举个例子：
一杯热水和一桶温水可能具有相同的温度（比如都是40°C），但由于水量不同，桶里的总热量更多。这说明温度不等于热量。

---

总结：  
**温度是衡量物体内部粒子热运动强弱的物理量，是热力学和日常生活中非常基础且重要的概念。**


OpenAI Python 的例子
```python
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "Orange."},
]
```
https://cookbook.openai.com/examples/how_to_format_inputs_to_chatgpt_models


但是langchain 需要使用以下的格式

In [13]:

from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
)


messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Knock knock."),
    AIMessage(content="Who's there?"),
    HumanMessage(content="Orange"),

]

In [17]:
# 第一次调用
res = chat.invoke(messages)
print(res)

content='Orange you glad I didn’t say banana? 😄🍌' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 45, 'total_tokens': 57, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-6fc657f9-f38e-928e-bf05-0b2425364762', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--6abfd148-a86d-494c-ad02-24932fcac67d-0' usage_metadata={'input_tokens': 45, 'output_tokens': 12, 'total_tokens': 57, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


因为 `res`也是`AIMessage`属性，所以我们可以直接进行添加，即可实现下一次的响应

In [18]:
# 如果你想把 AI 的回复继续加入对话
messages.append(res)

# 再次调用
res = chat.invoke(messages)
print(res.content)

Haha! 🍌🍊 That’s a classic twist!  
Orange you glad we’re not doing knock-knock jokes all day? 😄  
(But seriously, I love a good pun—keep ‘em coming!)


#### 处理LLM存在的缺陷
1. 容易出现幻觉
2. 信息滞后
3. 专业领域深度知识匮乏


In [19]:
messages = [
    SystemMessage(content="你是一个专业的知识助手。"),
    HumanMessage(content="你知道baichuan2模型吗？"),
]

In [21]:
res = chat.invoke(messages)
print(res.content)

是的，我知道Baichuan2模型。

Baichuan2是由百川智能（Baichuan Intelligent）推出的一系列开源大语言模型，是Baichuan系列的升级版本。它在多个方面进行了优化和增强，包括更大的训练数据量、更强的语言理解能力以及更好的多语言支持。Baichuan2模型有多个参数规模的版本（如7B、13B等），适用于不同的应用场景，从本地部署到大规模推理任务都能提供良好的性能。

Baichuan2的主要特点包括：

1. **大规模预训练**：基于海量互联网文本进行预训练，涵盖中文、英文等多种语言，具有较强的通用语言理解与生成能力。
2. **开源开放**：Baichuan2对研究和商业用途提供了不同程度的开源支持，促进了学术界和产业界的广泛应用。
3. **高效推理**：模型设计注重推理效率，在保持高性能的同时降低了资源消耗。
4. **支持微调**：支持通过LoRA、全参数微调等方式适配具体应用场景，比如对话系统、内容生成、代码生成等。
5. **中文优化**：相比一些国际主流模型，Baichuan2在中文语境下的表现尤为突出，更适合中文用户和本地化应用。

如果你有兴趣，可以访问百川智能的官方网站或其在Hugging Face、GitHub等平台的开源页面获取模型权重、技术文档和使用示例。

需要注意的是，使用该模型时应遵守相关的许可协议（如Baichuan2的许可证可能对商业用途有一定限制），建议仔细阅读官方发布的条款。

如果你有关于如何加载、微调或部署Baichuan2的问题，我也很乐意帮助！


chatgpt AI无法满足我们在某些特定领域的专业需求，我们可以通过知识注入的方式，利用prompt来解决这种问题：

In [22]:

baichuan2_information = [
    "Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。",
    "Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。",
    "Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。"
]

source_knowledge = "\n".join(baichuan2_information)

In [23]:
print(source_knowledge)

Baichuan 2是一个大规模多语言语言模型，它专注于训练在多种语言中表现优异的模型，包括不仅限于英文。这使得Baichuan 2在处理各种语言的任务时能够取得显著的性能提升。
Baichuan 2是从头开始训练的，使用了包括了2.6万亿个标记的庞大训练数据集。相对于以往的模型，Baichuan 2提供了更丰富的数据资源，从而能够更好地支持多语言的开发和应用。
Baichuan 2不仅在通用任务上表现出色，还在特定领域（如医学和法律）的任务中展现了卓越的性能。这为特定领域的应用提供了强有力的支持。


In [24]:
query = "你知道baichuan2模型吗？"

prompt_template = f"""基于以下内容回答问题：

内容:
{source_knowledge}

Query: {query}"""

In [26]:
prompt = HumanMessage(
    content=prompt_template
)
messages.append(prompt)

res = chat.invoke(messages)

In [27]:
print(res.content)

是的，我知道Baichuan 2模型。Baichuan 2是一个大规模多语言语言模型，专注于在多种语言中实现优异表现，不仅限于英文，因而在处理各种语言任务时能够显著提升性能。该模型是从头开始训练的，使用了包含2.6万亿个标记的庞大训练数据集，相比以往模型提供了更丰富的数据资源，更好地支持多语言的开发与应用。此外，Baichuan 2不仅在通用任务上表现出色，还在医学、法律等特定领域任务中展现了卓越的性能，为这些领域的专业应用提供了强有力的支持。


当我们注入一些专业的知识后，模型就能够很好的回答相关问题。
如果每一个问题都去用相关的外部知识进行增强拼接的话，那么回答的准确性就大大增加。

### 创建一个RAG对话模型

#### 1. 加载数据 （以baichuan2论文为例）

   https://arxiv.org/pdf/2309.10305v2.pdf

In [30]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("https://arxiv.org/pdf/2309.10305.pdf")

pages = loader.load_and_split()

In [31]:
pages[0]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-04-18T00:32:55+00:00', 'author': '', 'keywords': '', 'moddate': '2025-04-18T00:32:55+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'https://arxiv.org/pdf/2309.10305.pdf', 'total_pages': 28, 'page': 0, 'page_label': '1'}, page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Guo, Ruiyang Sun\nTao Zhang, Tianpeng Li, Tianyu Li, Wei Cheng, Weipeng Chen, Xiangrong Zeng\nXiaochuan W

#### 2. 知识切片 将文档分割成均匀的块。每个块是一段原始文本

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
)

docs = text_splitter.split_documents(pages)

In [35]:
print(len(docs), "chunks")
print(docs[0].page_content[:300])

216 chunks
Baichuan 2: Open Large-scale Language Models
Aiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan
Dian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai
Guosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji
Jian Xie, Juntao D


#### 3. 利用embedding模型对每个文本片段进行向量化，并储存到向量数据库中

此处默认采用阿里百炼的text-embedding-v1模型，前提是已经把阿里的api key配置到系统环境变量中

In [48]:
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS

# 1. 初始化 embedding 模型
embed_model = DashScopeEmbeddings(model="text-embedding-v1")

# 2. 构建 FAISS 向量数据库
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embed_model
)

ValueError: status_code: 401 
 code: InvalidApiKey 
 message: Invalid API-key provided.

#### 4. 通过向量相似度检索和问题最相关的K个文档。

In [ ]:
# 相似度检索
query = "How large is the Baichuan2 vocabulary?"
results = vectorstore.similarity_search(query, k=2)

for i, doc in enumerate(results, 1):
    print(f"--- Chunk {i} ---")
    print(doc.page_content[:300], "\n")

In [20]:
result

[Document(page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,', metadata={'page': 1, 'source': '/tmp/tmphxnlz_0y/tmp.pdf'}),
 Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong G

#### 5. 原始`query`与检索得到的文本组合起来输入到语言模型，得到最终的回答

In [21]:
def augment_prompt(query: str):
  # 获取top3的文本片段
  results = vectorstore.similarity_search(query, k=3)
  source_knowledge = "\n".join([x.page_content for x in results])
  # 构建prompt
  augmented_prompt = f"""Using the contexts below, answer the query.

  contexts:
  {source_knowledge}

  query: {query}"""
  return augmented_prompt

In [22]:
print(augment_prompt(query))

Using the contexts below, answer the query.

  contexts:
  languages, such as Chinese.
In this technical report, we introduce Baichuan
2, a series of large-scale multilingual language
models. Baichuan 2 has two separate models,
Baichuan 2-7B with 7 billion parameters and
Baichuan 2-13B with 13 billion parameters. Both
models were trained on 2.6 trillion tokens, which
to our knowledge is the largest to date, more than
double that of Baichuan 1 (Baichuan, 2023b,a).
With such a massive amount of training data,
Baichuan 2: Open Large-scale Language Models
Aiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan
Dian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai
Guosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji
Jian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma
Mang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Guo, Ruiyang Sun
adequate training of each word embedding. We
have 

In [23]:
# 创建prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)

messages.append(prompt)

res = chat(messages)

print(res.content)

Baichuan 2 has a vocabulary size of 125,696.


### 没有api key怎么办 创建一个本地的对话模型  


1.   embedding模型  
2.   chat模型


In [ ]:
! pip install sentence-transformers

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

model_name = "sentence-transformers/sentence-t5-large"

In [26]:
embedding = HuggingFaceEmbeddings(model_name=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

In [27]:
vectorstore_hf = Chroma.from_documents(documents=docs, embedding=embedding , collection_name="huggingface_embed")

In [28]:
query = "How large is the baichuan2 vocabulary?"

result = vectorstore_hf.similarity_search(query ,k = 2)


In [29]:
print(result)


[Document(page_content='languages, such as Chinese.\nIn this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,', metadata={'page': 1, 'source': '/tmp/tmphxnlz_0y/tmp.pdf'}), Document(page_content='Baichuan 2: Open Large-scale Language Models\nAiyuan Yang, Bin Xiao, Bingning Wang, Borong Zhang, Chao Yin, Chenxu Lv, Da Pan\nDian Wang, Dong Yan, Fan Yang, Fei Deng, Feng Wang, Feng Liu, Guangwei Ai\nGuosheng Dong, Haizhou Zhao, Hang Xu, Haoze Sun, Hongda Zhang, Hui Liu, Jiaming Ji\nJian Xie, Juntao Dai, Kun Fang, Lei Su, Liang Song, Lifeng Liu, Liyun Ru, Luyao Ma\nMang Wang, Mickel Liu, MingAn Lin, Nuolan Nie, Peidong Gu

通过本地部署的模型进行交互